In [32]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import ipywidgets
from scipy import constants as C
import scipy.interpolate
import IPython.display

mks_length = 0.8e-6/(2*np.pi)
#mks_length = 10.6e-6/(2*np.pi)
base_diagnostic = 'out/test'
uppe_list = glob.glob(base_diagnostic+'*_uppe_wave*.npy')
uppe_list += glob.glob(base_diagnostic+'*_uppe_source*.npy')
uppe_list += glob.glob(base_diagnostic+'*_uppe_plasma*.npy')
para_list = glob.glob(base_diagnostic+'*_paraxial_wave*.npy')
para_list += glob.glob(base_diagnostic+'*_paraxial_source*.npy')
para_list += glob.glob(base_diagnostic+'*_paraxial_plasma*.npy')
total_list = uppe_list + para_list

mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 24
l1_mm = 1e3*mks_length
t1_ps = 1e12*mks_length/C.c

file_w = ipywidgets.Dropdown(options=total_list,value=total_list[0],description='File')
frame_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,continuous_update=False)
xslice_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,continuous_update=False)
yslice_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,continuous_update=False)
tslice_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,continuous_update=False)
ax_w = ipywidgets.Dropdown(options=['Falsecolor-tx','Falsecolor-xy','Lineout'],value='Falsecolor-tx',description='Display')
rep_w = ipywidgets.Dropdown(options=['Time','Frequency','Wavelength'],value='Time',description='Representation')
color_w = ipywidgets.Dropdown(options=['viridis','gray','jet','plasma','inferno','ocean','seismic','bwr','prism',
                                      'nipy_spectral'],value='viridis',description='Color')
shape_w = ipywidgets.Textarea(value='no data',description='Shape')
bounds_w = ipywidgets.Textarea(value='no data',description='Bounds')
lim1_w = ipywidgets.FloatText(value=-1.0,description='Min')
lim2_w = ipywidgets.FloatText(value=1.0,description='Max')
transform_list = ['abs(x)','Log_e(abs(x))','Log_10(abs(x))','x^2','-x']
transform_select_w = ipywidgets.Dropdown(options=transform_list,value=transform_list[0],layout=ipywidgets.Layout(width='100px'))
transform_w = ipywidgets.Button(description='Transform')
autoscale_frame_w = ipywidgets.Button(description='Autoscale Frame')
autoscale_all_w = ipywidgets.Button(description='Autoscale All')

def load_data(file):
    global A,real_field,data_ext,qty_label
    real_field = file in uppe_list
    A = np.load(file)
    s = file.split('_')
    data_ext_name = s[0] + '_' + s[1] + '_' + s[2] + '_plot_ext.npy'
    data_ext = np.load(data_ext_name)
    data_ext = np.concatenate((data_ext[0:2],l1_mm*data_ext[2:4]))
    bounds_w.value = str(data_ext)
    shape_w.value = str(A.shape)
    frame_w.max = A.shape[3]-1
    tslice_w.max = A.shape[0]-1
    xslice_w.max = A.shape[1]-1
    yslice_w.max = A.shape[2]-1
    frame_w.value = 0
    tslice_w.value = np.int(A.shape[0]/2)
    xslice_w.value = np.int(A.shape[1]/2)
    yslice_w.value = np.int(A.shape[2]/2)
    if 'wave' in file:
        qty_label = ( r'$eA(t)/mc^2$' , r'$|E|^2(\omega)$' , r'$|E|^2(\lambda)$' )
    if 'source' in file:
        qty_label = ( r'$J(t)$' , r'$|J|(\omega)$' , r'$|J|(\lambda)$' )
    if 'plasma' in file:
        qty_label = ( r'$n_e(t)/n_c$' , r'$|n_e|(\omega)/n_c$' , r'$|n_e|(\lambda)/n_c$' )

def load(file):
    load_data(file)
    change_rep_data(rep_w.value)
    disp_frame(file,frame_w.value,ax_w.value,rep_w.value,color_w.value)

def get_freq_time_data(A,ext):
    if real_field:
        # Nodes are like [0,1,2,3] and walls are like [-0.5,0.5,1.5,2.5,3.5]
        # In these examples the user's requested upper bound would be 4 (it is thrown out)
        # Therefore element N/2+1 should be regarded as the central frequency
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        w_nodes = np.linspace(ext[0]+dw/2,ext[1]-dw/2,A.shape[0])
        return dw,wc,tmax,w_nodes
    else:
        # Nodes are like [-2,-1,0,1] and walls are like [-2.5,-1.5,-.5,.5,1.5]
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        w_nodes = np.linspace(ext[0]+dw/2,ext[1]-dw/2,A.shape[0])
        return dw,wc,tmax,w_nodes

def change_rep_data(rep):
    global plot_data,lbl4,plot_ext,qty_label
    plot_ext = np.copy(data_ext)
    if rep=='Time' and real_field:
        dw,wc,tmax,wn = get_freq_time_data(A,plot_ext)
        plot_ext[0] = 0.0
        plot_ext[1] = 1000*l1_mm*tmax
        #plot_ext[1] = t1_ps*tmax
        lbl4 = [r'$ct-z$ (um)',r'$x_1$ (mm)',r'$x_2$ (mm)',qty_label[0]]
        plot_data = np.fft.irfft(A,axis=0)[::-1,...]
    if rep=='Time' and not real_field:
        dw,wc,tmax,wn = get_freq_time_data(A,plot_ext)
        plot_ext[0] = 0.0
        plot_ext[1] = 1000*l1_mm*tmax
        lbl4 = [r'$ct-z$ (um)',r'$x_1$ (mm)',r'$x_2$ (mm)',qty_label[0]]
        plot_data = np.abs(np.fft.ifft(np.fft.ifftshift(A,axes=0),axis=0))[::-1,...]
    if rep=='Frequency':
        dw,wc,tmax,wn = get_freq_time_data(A,plot_ext)
        lbl4 = [r'$\omega/\omega_0$',r'$x_1$ (mm)',r'$x_2$ (mm)',qty_label[1]]
        plot_data = np.abs(A*wn[...,np.newaxis,np.newaxis,np.newaxis])**2
    if rep=='Wavelength':
        dw,wc,tmax,wn = get_freq_time_data(A,plot_ext)
        plot_data = (np.abs(A*wn[...,np.newaxis,np.newaxis,np.newaxis])**2)[1:][::-1]
        l_array_nu = (1000*l1_mm*2*np.pi/wn[1:])[::-1]
        l_array = np.linspace(l_array_nu[0],5,plot_data.shape[0])
        for i in range(plot_data.shape[1]):
            for j in range(plot_data.shape[2]):
                for k in range(plot_data.shape[3]):
                    fi = scipy.interpolate.interp1d(l_array_nu,plot_data[:,i,j,k])
                    plot_data[:,i,j,k] = fi(l_array)/l_array
        lbl4 = [r'$\lambda$ (um)',r'$x_1$ (mm)',r'$x_2$ (mm)',qty_label[2]]
        plot_ext[0] = l_array[0]
        plot_ext[1] = l_array[-1]

def change_rep(rep):
    change_rep_data(rep)
    disp_frame(file_w.value,frame_w.value,ax_w.value,rep,color_w.value)

def change_axes_data(ax):
    global plot_data
    if ax=='Falsecolor-tx':
        tslice_w.disabled = True
        xslice_w.disabled = True
        yslice_w.disabled = False
        yslice_w.max = plot_data.shape[2]-1
        yslice_w.value = np.int(plot_data.shape[2]/2)
    if ax=='Falsecolor-xy':
        tslice_w.disabled = False
        xslice_w.disabled = True
        yslice_w.disabled = True
        tslice_w.max = plot_data.shape[0]-1
        tslice_w.value = np.int(plot_data.shape[0]/2)
    if ax=='Lineout':
        tslice_w.disabled = True
        xslice_w.disabled = False
        yslice_w.disabled = False
        xslice_w.max = plot_data.shape[1]-1
        xslice_w.value = np.int(plot_data.shape[1]/2)        
        yslice_w.max = plot_data.shape[2]-1
        yslice_w.value = np.int(plot_data.shape[2]/2)        

def change_axes(ax):
    change_axes_data(ax)
    disp_frame(file_w.value,frame_w.value,ax_w.value,rep_w.value,color_w.value)

def transform_data(the_button):
    global plot_data,lbl4
    smallest_num = 1e-25
    if transform_select_w.value=='Log_e(abs(x))':
        lbl4[3] = r'$\log_{e}$' + lbl4[3]
        plot_data = np.log(smallest_num+np.abs(plot_data))
    if transform_select_w.value=='Log_10(abs(x))':
        lbl4[3] = r'$\log_{10}$' + lbl4[3]
        plot_data = np.log10(smallest_num+np.abs(plot_data))
    if transform_select_w.value=='x^2':
        plot_data = plot_data**2
    if transform_select_w.value=='-x':
        plot_data = -plot_data
    if transform_select_w.value=='abs(x)':
        plot_data = np.abs(plot_data)

def auto_scale_frame(the_button):
    t = tslice_w.value
    x = xslice_w.value
    y = yslice_w.value
    if ax_w.value=='Falsecolor-tx':
        lim1_w.value = np.min(plot_data[:,:,y,frame_w.value])
        lim2_w.value = np.max(plot_data[:,:,y,frame_w.value])
    if ax_w.value=='Falsecolor-xy':
        lim1_w.value = np.min(plot_data[t,:,:,frame_w.value])
        lim2_w.value = np.max(plot_data[t,:,:,frame_w.value])
    if ax_w.value=='Lineout':
        lim1_w.value = np.min(plot_data[:,x,y,frame_w.value])
        lim2_w.value = np.max(plot_data[:,x,y,frame_w.value])

def auto_scale_all(the_button):
    lim1_w.value = np.min(plot_data)
    lim2_w.value = np.max(plot_data)

def disp_frame(frame,t,x,y,color,lim2):
    global plot_data
    plt.figure(figsize=(20,10))
    if ax_w.value=='Lineout':
        hor = np.linspace(plot_ext[0],plot_ext[1],plot_data.shape[0])
        lineout = plt.plot(hor,plot_data[:,x,y,frame])
    if ax_w.value=='Falsecolor-xy':
        aspect_ratio = 'auto'
        plt.imshow(plot_data[t,:,:,frame].swapaxes(0,1),vmin=lim1_w.value,vmax=lim2_w.value,origin='lower',cmap=color,aspect=aspect_ratio,extent=plot_ext)
        b=plt.colorbar()
        plt.xlabel(lbl4[1],size=24)
        plt.ylabel(lbl4[2],size=24)
        b.set_label(lbl4[3],size=24)
    if ax_w.value=='Falsecolor-tx':
        aspect_ratio = 'auto'
        plt.imshow(plot_data[:,:,y,frame].swapaxes(0,1),vmin=lim1_w.value,vmax=lim2_w.value,origin='lower',cmap=color,aspect=aspect_ratio,extent=plot_ext)
        b=plt.colorbar()
        plt.xlabel(lbl4[0],size=24)
        plt.ylabel(lbl4[1],size=24)
        b.set_label(lbl4[3],size=24)
    plt.tight_layout()
    plt.show()

rep_iw = ipywidgets.interactive(change_rep,rep=rep_w)
ax_iw = ipywidgets.interactive(change_axes,ax=ax_w)
load_iw = ipywidgets.interactive(load,file=file_w)
update_iw = ipywidgets.interactive(disp_frame,frame=frame_w,t=tslice_w,x=xslice_w,y=yslice_w,color=color_w,lim2=lim2_w)
transform_w.on_click(transform_data)
autoscale_frame_w.on_click(auto_scale_frame)
autoscale_all_w.on_click(auto_scale_all)

autoscale_box = ipywidgets.HBox([autoscale_frame_w,autoscale_all_w])
transform_box = ipywidgets.HBox([transform_select_w,transform_w])
left_box = ipywidgets.VBox([file_w,frame_w,tslice_w,xslice_w,yslice_w,ax_w,rep_w,color_w,shape_w,bounds_w,lim1_w,lim2_w,transform_box,autoscale_box],
                            layout=ipywidgets.Layout(width='330px',flex='0 0 auto'))


fig_pane = update_iw.children[-1]
fig_pane.layout.flex = '1 1 auto'
main_view = ipywidgets.HBox([left_box, fig_pane],
                            layout=ipywidgets.Layout(width='100%',height='700px',display='flex',
                                                     align_items='stretch',
                                                     align_content='flex-start',
                                                     justify_content='flex-start'))

load_data(file_w.value)
change_rep_data(rep_w.value)
change_axes_data(ax_w.value)
IPython.display.display(main_view)